<span type="title">Spring MVC</span> | <span type="update">2018-10-25</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍了 Spring MVC 的基本使用。</p></span>

# Spring MVC 概述

Spring MVC 是一套用来代替 Struts 的 Servlet 框架。相比较传统的 Servlet 技术，基于 Struts 的框架设计模式采用一个 Super Servlet 对所有的 URL 请求进行转发和分派，然后再交给子 Servlet 进行请求，之后由 JSP 等模板技术完成渲染工作。这套框架很好的避免了大工程需要繁琐的配置 web.xml 文件、同时能够对于 URL 请求进行更轻松的解析和处理。Spring MVC 基于 Spring IOC 容器，是 Struts 的替代产品，主要用来进行 Controller 层的控制，完成 View 层和 Model 层之间的交互。

其基本作用原理是：使用 Spring 内置的 Servlet 拦截所有请求，交给自家的 Handler 完成处理，之后再委托 JSP 等模板引擎进行渲染。需要配置 web.xml 文件完成拦截和监听，配置 spring-mvc.xml 等配置文件来具体定义 Handler 的查找路径、模板引擎的处理器等等。

Spring MVC 相比较 Struts 的优点在于：天生和 Spring 框架集成，支持 Restful 风格的开发，从 URL 到映射的处理十分方便，数据验证、格式化和绑定功能强大、好用，对于异常的处理支持较好，对于静态资源文件提供了支持。



# Hello World in Spring MVC

Web.xml 文件需要配置如下的 Servlet，完成拦截，需要指明类为 DispatcherServlet，此外，需要传入 Spring MVC 的 xml 格式配置，放置在 Servlet 的上下文中，名称为 ContextConfigLocation，如果这里没有配置，则默认寻找 `/WEB-INF/{serlvet-name}-servlet.xml` 文件作为配置。

```xml
<servlet>
    <servlet-name>springDispatcherServlet</servlet-name>
    <servlet-class>org.springframework.web.servlet.DispatcherServlet</servlet-class>
    <init-param>
        <param-name>contextConfigLocation</param-name>
        <param-value>springmvc.cfg.xml</param-value>
    </init-param>
</servlet>
<servlet-mapping>
    <servlet-name>springDispatcherServlet</servlet-name>
    <url-pattern>/</url-pattern>
</servlet-mapping>
```

为了支持 Restful 风格 URL 格式，推荐添加如下过滤器, HiddenHttpMethodFilter 过滤器可以对请求进行过滤，根据隐藏字段，将 POST, GET 请求转换成 POST, GET, PUT, DELETE 四种原生的 HTTP 方法。

```xml
<filter>
    <filter-name>methodFilter</filter-name>
    <filter-class>org.springframework.web.filter.HiddenHttpMethodFilter</filter-class>
</filter>
<filter-mapping>
    <filter-name>methodFilter</filter-name>
    <url-pattern>/*</url-pattern>
</filter-mapping>
```

Ps. 使用 IDEA 生成的 web.xml 有时候会莫名其妙的导致 404，无法找到 Spring Servlet，但是 jsp 页面可以直接访问（所以肯定是 web.xml 的锅，但是这个文件确实能够找到，没有报找不到 web.xml 的错误，但就是解析不了里面的内容），这时候新建一个 web.xml，然后将之前设置复制进来即可。

一个基本的 springmvc.cfg.xml 配置文件如下：

```xml
<?xml version="1.0" encoding="UTF-8"?>
<beans xmlns="http://www.springframework.org/schema/beans"
       xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
       xmlns:beans="http://www.springframework.org/schema/beans"
       xmlns:context="http://www.springframework.org/schema/context"
       xmlns:mvc="http://www.springframework.org/schema/mvc"
       xsi:schemaLocation="http://www.springframework.org/schema/beans
       http://www.springframework.org/schema/beans/spring-beans.xsd
       http://www.springframework.org/schema/context
       http://www.springframework.org/schema/context/spring-context.xsd
       http://www.springframework.org/schema/mvc
       http://www.springframework.org/schema/mvc/spring-mvc.xsd
">
    <context:component-scan base-package="com.mazhangjing.mvc.hello"/>
    <bean id="viewResolver"
          class="org.springframework.web.servlet.view.InternalResourceViewResolver">
        <property name="prefix" value="/WEB-INF/views/" />
        <property name="suffix" value=".jsp" />
    </bean>

    <!--配置自定义的 Viewer，需要添加 @Component 注解，需要添加 order 优先级来指定优先级，数值越小，则越优先。-->
    <bean class="org.springframework.web.servlet.view.BeanNameViewResolver">
        <property name="order" value="100"></property>
    </bean>

    <!--添加 JSTL 标签库，自动使用 JSTLView，可以通过配置 messageSource 来指定国际化文件配置
    在 JSP 页面中启用 taglib 的 fmt:message 来设置 key。-->
    <bean id="messageSource"
          class="org.springframework.context.support.ResourceBundleMessageSource">
        <property name="basename" value="i18n"></property>
    </bean>
    <!--或者可以不经过 Handler，直接完成 View-Controller 和 View 的对应。此种情况下需要添加 mvc 注解驱动-->
    <mvc:view-controller path="/success" view-name="success" />
    <mvc:annotation-driven />

</beans>
```

注意这里面较为重要的一些设置：

- 启用 context